In [ ]:
!pip install kaggle
!kaggle datasets list


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 398, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
# Step 3: Configure Kaggle
import os
os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/

# Step 4: Download the dataset
!kaggle datasets download -d nih-chest-xrays/sample

# Step 5: Unzip the dataset
!unzip sample.zip

Streaming output truncated to the last 5000 lines.
  inflating: sample/sample/images/00003124_000.png  
  inflating: sample/sample/images/00003129_000.png  
  inflating: sample/sample/images/00003129_001.png  
  inflating: sample/sample/images/00003129_004.png  
  inflating: sample/sample/images/00003129_009.png  
  inflating: sample/sample/images/00003129_011.png  
  inflating: sample/sample/images/00003135_000.png  
  inflating: sample/sample/images/00003140_007.png  
  inflating: sample/sample/images/00003159_001.png  
  inflating: sample/sample/images/00003169_001.png  
  inflating: sample/sample/images/00003177_002.png  
  inflating: sample/sample/images/00003186_004.png  
  inflating: sample/sample/images/00003186_005.png  
  inflating: sample/sample/images/00003190_018.png  
  inflating: sample/sample/images/00003191_001.png  
  inflating: sample/sample/images/00003198_002.png  
  inflating: sample/sample/images/00003198_005.png  
  inflating: sample/sample/images/00003198_009.p

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import os
import zipfile
import shutil

# Define paths
zip_path = 'sample.zip'
extract_to = '/content/'
image_folder_name = 'chest_xray_images'
csv_path = 'sample/sample_labels.csv'

# Function to unzip and extract images
def unzip_and_extract_images(zip_path, extract_to, image_folder_name):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Extract all contents to a temporary directory
        temp_dir = os.path.join(extract_to, 'temp_extracted')
        zip_ref.extractall(temp_dir)

        # Move image files to a new directory
        images_dir = os.path.join(extract_to, image_folder_name)
        os.makedirs(images_dir, exist_ok=True)

        for root, _, files in os.walk(os.path.join(temp_dir, 'sample/images')):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    shutil.move(os.path.join(root, file), os.path.join(images_dir, file))

        # Clean up the temporary directory
        shutil.rmtree(temp_dir)

# Unzip and extract images
unzip_and_extract_images(zip_path, extract_to, image_folder_name)

In [ ]:
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Load labels
labels_df = pd.read_csv(csv_path)

# Create label mapping with class names as strings
label_map = {label: idx for idx, label in enumerate(labels_df['Finding Labels'].unique())}
reverse_label_map = {v: k for k, v in label_map.items()}
labels_df['label'] = labels_df['Finding Labels'].map(label_map).astype(str)  # Convert labels to strings

# Prepare data generators
batch_size = 32
img_height, img_width = 224, 224

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    rotation_range=20
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory=os.path.join(extract_to, image_folder_name),
    x_col='Image Index',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory=os.path.join(extract_to, image_folder_name),
    x_col='Image Index',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Define the ResNet model
def create_resnet_model(num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    # Unfreeze some top layers for fine-tuning
    for layer in base_model.layers[-20:]:  # Unfreeze the last 20 layers
        layer.trainable = True

    return model

# Create the model
num_classes = len(label_map)
model = create_resnet_model(num_classes)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,  # Increase epochs if needed
    verbose=1
)

# Save the model
model.save('/content/chest_xray.h5')


Found 4485 validated image filenames belonging to 244 classes.
Found 1121 validated image filenames belonging to 244 classes.
94765736/94765736 [==============================] - 3s 0us/step
Epoch 1/20
141/141 [==============================] - 4392s 31s/step - loss: 2.5984 - accuracy: 0.5231 - val_loss: 3.2634 - val_accuracy: 0.0473
Epoch 2/20
141/141 [==============================] - 4289s 30s/step - loss: 2.2815 - accuracy: 0.5336 - val_loss: 3.6341 - val_accuracy: 0.5575
Epoch 3/20
 10/141 [=>............................] - ETA: 1:03:21 - loss: 2.1053 - accuracy: 0.5500

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the trained model
model = load_model('/content/resnet_chest_xray.h5')

# Function to preprocess the image and make a prediction
def predict_disease(image_path):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale the image

    # Predict the class
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]  # Get the class index

    # Print raw predictions for debugging
    print(f'Raw predictions: {predictions}')

    # Map the class index to the label
    predicted_label = reverse_label_map[predicted_class]

    return predicted_label

# Example usage
new_image_path = '/content/emphysema.jpeg'
predicted_disease = predict_disease(new_image_path)
print(f'The predicted disease is: {predicted_disease}')


In [ ]:
# Load labels
labels_df = pd.read_csv(csv_path)

# Create label mapping with class names as strings
label_map = {label: idx for idx, label in enumerate(labels_df['Finding Labels'].unique())}
reverse_label_map = {v: k for k, v in label_map.items()}
labels_df['label'] = labels_df['Finding Labels']

# Prepare data generators
batch_size = 32
img_height, img_width = 224, 224

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    rotation_range=20
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory=os.path.join(extract_to, image_folder_name),
    x_col='Image Index',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory=os.path.join(extract_to, image_folder_name),
    x_col='Image Index',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Define the ResNet model
def create_resnet_model(num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    return model

# Create the model
num_classes = len(label_map)
model = create_resnet_model(num_classes)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    verbose=1
)

# Save the model
model.save('/content/resnet_chest_xray.h5')

Found 4485 validated image filenames belonging to 244 classes.
Found 1121 validated image filenames belonging to 244 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


141/141 ━━━━━━━━━━━━━━━━━━━━ 190s 1s/step - accuracy: 0.4668 - loss: 3.3735 - val_accuracy: 0.5682 - val_loss: 2.3292
Epoch 2/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 164s 1s/step - accuracy: 0.5446 - loss: 2.4378 - val_accuracy: 0.5682 - val_loss: 2.3530
Epoch 3/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - accuracy: 0.5376 - loss: 2.4508 - val_accuracy: 0.5682 - val_loss: 2.3488
Epoch 4/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 205s 1s/step - accuracy: 0.5313 - loss: 2.4422 - val_accuracy: 0.5682 - val_loss: 2.3598
Epoch 5/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 203s 1s/step - accuracy: 0.5355 - loss: 2.4434 - val_accuracy: 0.5682 - val_loss: 2.3626
Epoch 6/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 208s 1s/step - accuracy: 0.5382 - loss: 2.4369 - val_accuracy: 0.5682 - val_loss: 2.3547
Epoch 7/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 162s 1s/step - accuracy: 0.5389 - loss: 2.4225 - val_accuracy: 0.5682 - val_loss: 2.3678
Epoch 8/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 190s 1s/step - accuracy: 0.5447 - loss: 2.4260 - val_accuracy: 0.568

In [ ]:
model.evaluate(validation_generator)

36/36 ━━━━━━━━━━━━━━━━━━━━ 32s 878ms/step - accuracy: 0.5557 - loss: 2.4586


[2.3926100730895996, 0.5682426691055298]